# Imports

In [1]:
import xgboost as xgb
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

# Train

In [19]:
train_set = pd.read_csv('train/train_encoded.csv')
train_set.head(10)

,id,len_keyword,len_location,len_text,len_text_original,diff_len_text,keyword_sum,keyword_min,keyword_max,keyword_mean,...,text_min_tf-idf,text_max_tf-idf,text_mean_tf-idf,text_median_tf-idf,text_sum_w2v,text_min_w2v,text_max_w2v,text_mean_w2v,text_median_w2v,target
0,1,0.0,0.0,68,69,1,1.0,1.0,1.0,1.0,...,0.099866,0.430347,0.257765,0.249176,11.683651,-0.018116,0.615265,0.177025,0.150866,1
1,4,0.0,0.0,38,38,0,1.0,1.0,1.0,1.0,...,0.227820,0.504613,0.365052,0.347656,0.790154,-0.103024,0.308946,0.079015,0.055061,1
2,5,0.0,0.0,131,133,2,1.0,1.0,1.0,1.0,...,0.074545,0.497279,0.232038,0.219919,32.468316,-0.044882,1.000000,0.154611,0.113670,1
3,6,0.0,0.0,63,65,2,1.0,1.0,1.0,1.0,...,0.128410,0.484991,0.335797,0.356367,1.943116,-0.039545,0.317247,0.092529,0.078856,1
4,7,0.0,0.0,86,88,2,1.0,1.0,1.0,1.0,...,0.096273,0.390788,0.248369,0.241596,13.153737,-0.041156,1.000000,0.125274,0.086994,1
5,8,0.0,0.0,105,110,5,1.0,1.0,1.0,1.0,...,0.089499,0.363373,0.238811,0.245703,6.677405,-0.093557,0.478663,0.101173,0.080587,1
6,10,0.0,0.0,92,95,3,1.0,1.0,1.0,1.0,...,0.096857,0.393245,0.255461,0.272769,9.341228,-0.047981,0.788929,0.141534,0.109943,1
7,13,0.0,0.0,60,59,-1,1.0,1.0,1.0,1.0,...,0.128963,0.487079,0.246268,0.228644,12.705849,-0.018808,1.000000,0.162896,0.129083,1
8,14,0.0,0.0,80,79,-1,1.0,1.0,1.0,1.0,...,0.131362,0.399979,0.272497,0.254940,15.609978,-0.033696,1.000000,0.200128,0.158737,1
9,15,0.0,0.0,53,52,-1,1.0,1.0,1.0,1.0,...,0.130843,0.507948,0.277341,0.259876,7.238668,0.004347,0.526257,0.160859,0.136866,1


In [20]:
train_set.drop('id', axis=1, inplace=True)

### XGBoost

In [21]:
X, y = train_set.iloc[:,:-1],train_set.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

### Random Search

In [5]:
learning_rate = [0.1, 0.3, 0.01, 0.001]
max_depth = [20, 50, 70, 100]
#subsamples = [0.4, 0.5, 0.6]
colsample_bytree = [0.4, 0.5, 0.6]
n_estimators = [20, 50, 70, 100]
objective = ['reg:logistic']

random_search = {'learning_rate': learning_rate, 'max_depth': max_depth,\
                'colsample_bytree': colsample_bytree, 'n_estimators': n_estimators, 'objective': objective}

In [6]:
xgb_random = RandomizedSearchCV(estimator = xgb.XGBClassifier(),\
                               param_distributions=random_search, n_iter=50, cv=7, random_state=111)

In [7]:
xgb_random.fit(X_train, y_train)

RandomizedSearchCV(cv=7,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n_estimators=100,...
                                           num_parallel_tree=None,
                                           random_state=None, reg_alpha=None,
                                    

In [8]:
xgb_random.best_params_

{'objective': 'reg:logistic',
 'n_estimators': 100,
 'max_depth': 70,
 'learning_rate': 0.1,
 'colsample_bytree': 0.4}

In [ ]:
preds = xgb_random.predict(X_test)
f1 = np.sqrt(f1_score(y_test, preds))
print("F1-Mean Score: %f" % (f1))

In [24]:
xg_reg = xgb.XGBClassifier(objective ='reg:logistic', colsample_bytree = 0.4,\
                           learning_rate = 0.001, max_depth = 50, n_estimators = 50)
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
f1 = np.sqrt(f1_score(y_test, preds))
print("F1-Mean Score: %f" % (f1))

F1-Mean Score: 0.780995


In [25]:
xg_reg = xgb.XGBClassifier(objective ='reg:logistic', colsample_bytree = 0.5,\
                           learning_rate = 0.1, max_depth = 100, n_estimators = 100)
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
f1 = np.sqrt(f1_score(y_test, preds))
print("F1-Mean Score: %f" % (f1))

F1-Mean Score: 0.783653


In [26]:
xg_reg = xgb.XGBClassifier(objective ='reg:logistic', colsample_bytree = 0.5,\
                           learning_rate = 0.01, max_depth = 100, n_estimators = 100)
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
f1 = np.sqrt(f1_score(y_test, preds))
print("F1-Mean Score: %f" % (f1))

F1-Mean Score: 0.780693


##### Best params de random forest

In [27]:
xg_reg = xgb.XGBClassifier(objective ='reg:logistic', subsample=0.3, colsample_bytree = 0.6,\
                           learning_rate = 0.1, max_depth = 70, alpha = 10, n_estimators = 100)
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
f1 = np.sqrt(f1_score(y_test, preds))
print("F1-Mean Score: %f" % (f1))

F1-Mean Score: 0.773945


##### Parámetros que mejor funcionaron con pruebas a mano

In [28]:
xg_reg = xgb.XGBClassifier(objective ='reg:logistic', subsample=0.3,
                colsample_bytree = 0.6, learning_rate = 0.1,
                max_depth = 70, alpha = 10, n_estimators = 57)
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
f1 = np.sqrt(f1_score(y_test, preds))
print("F1-Mean Score: %f" % (f1))

F1-Mean Score: 0.773956


# Test

In [16]:
test_set = pd.read_csv('test/test_encoded.csv')

In [17]:
col_id = test_set['id']
test_set.drop('id', axis=1, inplace=True)

In [29]:
preds = xg_reg.predict(test_set)
preds

array([0, 0, 0, ..., 0, 0, 1])

In [ ]:
test_set['target'] = preds
test_set['id'] = col_id
test_set[['id', 'target']]

### Resultado

In [30]:
test_set[['id', 'target']].to_csv('result_xgb.csv', header=True, index=False)